In [135]:
import numpy as np 
import cv2
import os
from tqdm import tqdm
import xml.etree.ElementTree as ET
import json
import shutil
from PIL import Image

In [ ]:
# 1.stop, 2.railroad 3.yield 4.traffic light

In [2]:
# data format: [(x_max + x_min)/2,(y_max + y_min)/2,x_max - x_min,y_max - y_min])]

In [144]:
label_list = os.listdir(r'D:\TrafficSign\Mapillary\Data\Labels_S')
image_list = os.listdir(r'D:\TrafficSign\Mapillary\Data\Images_S')

In [148]:
label_name = label_list[20]
image_name = label_name.replace('txt','jpg')
image = cv2.imread(os.path.join(r'D:\TrafficSign\Mapillary\Data\Images_S',image_name))
file_path = os.path.join(r'D:\TrafficSign\Mapillary\Data\Labels_S',label_name)
with open(file_path, 'r') as f:
    lines = f.readlines()
lines = [line.strip() for line in lines]

for i in range(len(lines)):
    label_test = lines[i].split(' ')
    bbox = np.array(label_test[1:]).astype(np.float32)
    x1 = int((bbox[0] - 0.5 * bbox[2]) * image.shape[1])
    y1 = int((bbox[1] - 0.5 * bbox[3]) * image.shape[0])
    x2 = int((bbox[0] + 0.5 * bbox[2]) * image.shape[1])
    y2 = int((bbox[1] + 0.5 * bbox[3]) * image.shape[0])
    cv2.rectangle(image, (x1, y1), (x2, y2), (255,0,0), 5)
image = cv2.resize(image, (1024, 1024))
cv2.imshow('a',image)
cv2.waitKey(0) # 0==wait forever


-1

In [131]:
label_list = os.listdir(r'D:\TrafficSign\archive\annotations')
image_list = os.listdir(r'D:\TrafficSign\archive\images')

In [132]:
Label_target_path = 'D:\TrafficSign\Mapillary\Data\Labels_S'
Image_target_path = 'D:\TrafficSign\Mapillary\Data\Images_S'

In [ ]:
list_dir = os.listdir(Image_target_path)
for i in tqdm(range(len(list_dir))):
    if list_dir[i].split('.')[-1] == 'png':
        im1 = Image.open(os.path.join(Image_target_path,list_dir[i]))
        im1 = im1.convert('RGB')
        im1.save(os.path.join(Image_target_path,list_dir[i].replace('png','jpg')))

In [155]:
labels = []
bboxes = []
for ind in tqdm(range(len(label_list))):
#     image = cv2.imread(os.path.join(r'D:\TrafficSign\archive\images',image_list[ind]))
    image = Image.open(os.path.join(r'D:\TrafficSign\archive\images',image_list[ind]))
    image = image.convert('RGB')
    tree = ET.parse(os.path.join(r'D:\TrafficSign\archive\annotations',label_list[ind]))
    root = tree.getroot()
    for ob in root:
        if ob.tag == 'object':
            xmin,ymin,xmax,ymax = int(ob[5][0].text),int(ob[5][1].text),int(ob[5][2].text),int(ob[5][3].text)
            coord = [((xmax + xmin)/2)/image.shape[0],((ymax + ymin)/2)/image.shape[1],(xmax - xmin)/image.shape[0],(ymax - ymin)/image.shape[1]]
            coord = np.array(coord).astype('str')
            labels.append(ob[0].text)

100%|███████████████████████████████████████████████████████████████████████████████| 877/877 [00:03<00:00, 241.07it/s]


In [143]:
# ind = 6
labels = []
bboxes = []
for ind in tqdm(range(len(label_list))):
    image = Image.open(os.path.join(r'D:\TrafficSign\archive\images',image_list[ind]))
    image = image.convert('RGB')
    tree = ET.parse(os.path.join(r'D:\TrafficSign\archive\annotations',label_list[ind]))
    root = tree.getroot()
    interested_flag = False
    for ob in root:
        if ob.tag == 'object':
            if ob[0].text == 'trafficlight':
                interested_flag = True
                break
        if interested_flag:
            break
    if interested_flag:
#         shutil.copy(os.path.join(r'D:\TrafficSign\archive\images',image_list[ind]), os.path.join(Image_target_path,image_list[ind]))
        image.save(os.path.join(Image_target_path,image_list[ind].replace('png','jpg')))
        with open(os.path.join(Label_target_path,image_list[ind].split('.')[0]+'.txt'), 'w') as label_file:
            for ob in root:
                if ob.tag == 'object':
                    xmin,ymin,xmax,ymax = int(ob[5][0].text),int(ob[5][1].text),int(ob[5][2].text),int(ob[5][3].text)
                    coord = [((xmax + xmin)/2)/image.size[0],((ymax + ymin)/2)/image.size[1],(xmax - xmin)/image.size[0],(ymax - ymin)/image.size[1]]
                    coord = np.array(coord).astype('str')
                    class_ind = 0
                    label = ob[0].text
                    if label == 'stop':
                        class_ind = 0
                        label_file.write('{} '.format(class_ind) + ' '.join(coord))
                        label_file.write('\n')
                    elif label == 'trafficlight':
                        class_ind = 3
                        label_file.write('{} '.format(class_ind) + ' '.join(coord))
                        label_file.write('\n')
    

100%|████████████████████████████████████████████████████████████████████████████████| 877/877 [00:20<00:00, 42.39it/s]


## Mapillary Prep

In [ ]:
# 1.stop, 2.railroad 3.yield 4.traffic light

In [128]:
label_path = r'D:\TrafficSign\Mapillary\mtsd_v2_fully_annotated\annotations'
image_path = r'D:\TrafficSign\Mapillary\images'
label_list = os.listdir(label_path)
image_list = os.listdir(image_path)

In [129]:
Label_target_path = 'D:\TrafficSign\Mapillary\Data\Labels'
Image_target_path = 'D:\TrafficSign\Mapillary\Data\Images'
Targets = ['regulatory--stop', 'warning--railroad-crossing', 'regulatory--yield']


In [5]:
i = 13
f = image_list[i]
label_name = f.split('.')[0]+'.json'
with open(os.path.join(label_path,label_name), 'r') as j:
    anno = json.loads(j.read())
target_image_path = os.path.join(image_path,f)
image = cv2.imread(target_image_path)
for ind in range(len(anno['objects'])):
    label = anno['objects'][ind]['label']
    interested_flag = False
    for class_ind,t in enumerate(Targets):
        if t in label:
            interested_flag = True
            break
    if interested_flag:
        break
if interested_flag:
#         print('yes')
    shutil.copy(target_image_path, os.path.join(Image_target_path,f))
    with open(os.path.join(Label_target_path,f.split('.')[0]+'.txt'), 'w') as label_file:
        for ind in range(len(anno['objects'])):
            label = anno['objects'][ind]['label']
            interested_flag = False
            for class_ind,t in enumerate(Targets):
                if t in label:
                    interested_flag = True
                    break
            if interested_flag:
                bbox = anno['objects'][ind]['bbox']
                xmin,ymin,xmax,ymax = int(bbox['xmin']),int(bbox['ymin']),int(bbox['xmax']),int(bbox['ymax'])
                coord = [((xmax + xmin)/2)/image.shape[1],((ymax + ymin)/2)/image.shape[0],(xmax - xmin)/image.shape[1],(ymax - ymin)/image.shape[0]]
                coord = np.array(coord).astype('str')
                label_file.write('{} '.format(class_ind) + ' '.join(coord))
                label_file.write('\n')

In [130]:
for i,f in tqdm(enumerate(image_list)):
    label_name = f.split('.')[0]+'.json'
    if label_name not in label_list:
        continue
    with open(os.path.join(label_path,label_name), 'r') as j:
        anno = json.loads(j.read())
    target_image_path = os.path.join(image_path,f)
    image = cv2.imread(target_image_path)
    for ind in range(len(anno['objects'])):
        label = anno['objects'][ind]['label']
        interested_flag = False
        for class_ind,t in enumerate(Targets):
            if (t in label)&(label.split('--')[-1]=='g1'):
                interested_flag = True
                break
        if interested_flag:
            break
    if interested_flag:
#         print('yes')
        shutil.copy(target_image_path, os.path.join(Image_target_path,f))
        with open(os.path.join(Label_target_path,f.split('.')[0]+'.txt'), 'w') as label_file:
            for ind in range(len(anno['objects'])):
                label = anno['objects'][ind]['label']
                interested_flag = False
                for class_ind,t in enumerate(Targets):
                    if (t in label)&(label.split('--')[-1]=='g1'):
                        interested_flag = True
                        break
                if interested_flag:
#                     print(label,i)
                    bbox = anno['objects'][ind]['bbox']
                    xmin,ymin,xmax,ymax = int(bbox['xmin']),int(bbox['ymin']),int(bbox['xmax']),int(bbox['ymax'])
                    coord = [((xmax + xmin)/2)/anno['width'],((ymax + ymin)/2)/anno['height'],(xmax - xmin)/anno['width'],(ymax - ymin)/anno['height']]
                    coord = np.array(coord).astype('str')
                    label_file.write('{} '.format(class_ind) + ' '.join(coord))
                    label_file.write('\n')


13344it [15:44, 14.13it/s]


# Vis

In [108]:
image_ind = 500
label_name = image_list[image_ind].split('.')[0]+'.json'
with open(os.path.join(label_path,label_name), 'r') as j:
    anno = json.loads(j.read())
image = cv2.imread(os.path.join(image_path,image_list[image_ind]))
for ind in range(len(anno['objects'])):
    bbox = anno['objects'][ind]['bbox']
    xmin,ymin,xmax,ymax = int(bbox['xmin']),int(bbox['ymin']),int(bbox['xmax']),int(bbox['ymax'])
    cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (255,0,0), 5)
image = cv2.resize(image, (1024, 1024))
cv2.imshow('a',image)
cv2.waitKey(0) # 0==wait forever

-1

# Create Dataset

In [1]:
from PIL import Image

# im1 = Image.open(r'path where the PNG is stored\file name.png')
# im1.save(r'path where the JPG will be stored\new file name.jpg')

In [4]:
Label_target_path = 'D:\TrafficSign\Mapillary\Data\Labels'
Image_target_path = 'D:\TrafficSign\Mapillary\Data\Images'

In [9]:
list_dir = os.listdir(Image_target_path)
for i in tqdm(range(len(list_dir))):
    if list_dir[i].split('.')[-1] == 'png':
        im1 = Image.open(os.path.join(Image_target_path,list_dir[i]))
        im1 = im1.convert('RGB')
        im1.save(os.path.join(Image_target_path,list_dir[i].replace('png','jpg')))

100%|████████████████████████████████████████████████████████████████████████████| 5617/5617 [00:01<00:00, 4368.29it/s]


In [149]:
image_path = 'D:\TrafficSign\Mapillary\Data\Images'
image_list = np.array(os.listdir(image_path))
label_path = 'D:\TrafficSign\Mapillary\Data\Labels'
label_list = np.array(os.listdir(label_path))

In [150]:
shuffle_ind = np.random.choice(np.arange(len(image_list)),size = (len(image_list)),replace = False)

In [151]:
image_list = image_list[shuffle_ind]
label_list = label_list[shuffle_ind]

In [152]:
split_ratio = [7,2,1] # train,val,test

In [153]:
train_len = int(len(image_list)*0.7)
val_len = int(len(image_list)*0.2)
test_len = len(image_list) - train_len - val_len

In [154]:
for i in tqdm(range(train_len)):
    shutil.copy(os.path.join(image_path,image_list[i]), os.path.join(r'D:\TrafficSign\Mapillary\SplitedData\images\train',image_list[i]))
    shutil.copy(os.path.join(label_path,label_list[i]), os.path.join(r'D:\TrafficSign\Mapillary\SplitedData\labels\train',label_list[i]))

100%|██████████████████████████████████████████████████████████████████████████████| 3342/3342 [02:06<00:00, 26.32it/s]


In [155]:
for i in tqdm(range(train_len,train_len+val_len)):
    shutil.copy(os.path.join(image_path,image_list[i]), os.path.join(r'D:\TrafficSign\Mapillary\SplitedData\images\val',image_list[i]))
    shutil.copy(os.path.join(label_path,label_list[i]), os.path.join(r'D:\TrafficSign\Mapillary\SplitedData\labels\val',label_list[i]))

100%|████████████████████████████████████████████████████████████████████████████████| 955/955 [00:38<00:00, 24.74it/s]


In [156]:
for i in tqdm(range(train_len+val_len,len(image_list))):
    shutil.copy(os.path.join(image_path,image_list[i]), os.path.join(r'D:\TrafficSign\Mapillary\SplitedData\images\test',image_list[i]))
    shutil.copy(os.path.join(label_path,label_list[i]), os.path.join(r'D:\TrafficSign\Mapillary\SplitedData\labels\test',label_list[i]))

100%|████████████████████████████████████████████████████████████████████████████████| 478/478 [00:19<00:00, 24.29it/s]


# Resize

In [167]:
tar_path = r'D:\TrafficSign\custom_dataset\images\test'
img_list = os.listdir(tar_path)
for i in tqdm(range(len(img_list))):
    image = cv2.imread(os.path.join(tar_path,img_list[i]))
    image = cv2.resize(image, (1280, 1280))
    cv2.imwrite(os.path.join(tar_path,img_list[i]),image)

100%|████████████████████████████████████████████████████████████████████████████████| 478/478 [00:41<00:00, 11.39it/s]


In [ ]:
image = cv2.imread(os.path.join(r'D:\TrafficSign\Mapillary\Data\Images_S',image_name))

In [163]:
label_name = label_list[1254]
image_name = label_name.replace('txt','jpg')
image = cv2.imread(os.path.join(r'D:\TrafficSign\custom_dataset\images\train',image_name))
image = cv2.resize(image, (1280, 1280))
file_path = os.path.join(r'D:\TrafficSign\custom_dataset\labels\train',label_name)
with open(file_path, 'r') as f:
    lines = f.readlines()
lines = [line.strip() for line in lines]

for i in range(len(lines)):
    label_test = lines[i].split(' ')
    bbox = np.array(label_test[1:]).astype(np.float32)
    x1 = int((bbox[0] - 0.5 * bbox[2]) * image.shape[1])
    y1 = int((bbox[1] - 0.5 * bbox[3]) * image.shape[0])
    x2 = int((bbox[0] + 0.5 * bbox[2]) * image.shape[1])
    y2 = int((bbox[1] + 0.5 * bbox[3]) * image.shape[0])
    cv2.rectangle(image, (x1, y1), (x2, y2), (255,0,0), 3)

cv2.imshow('a',image)
cv2.waitKey(0) # 0==wait forever


-1

In [164]:
cv2.imwrite(r'D:\TrafficSign\test.jpg',image)

True